In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyLDAvis 
!python -m spacy download pt

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
import ast
nltk.download('stopwords') 
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from pyLDAvis import sklearn as sklearn_lda
import re
import os
import numpy as np
from gensim import models, corpora
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from nltk import tokenize
from tqdm.notebook import tqdm
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
doc = spacy.load('pt')

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projeto Juliana/Notebooks/Juarez_words_full.csv')

In [ ]:
df = df[df['Porcentagem'] > 25]

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/Projeto Juliana/Notebooks/Juarez_words_full.csv')
df_words = df[['Palavras']]
df_words['Palavras'] = df_words['Palavras'].apply(ast.literal_eval)


stop_words = stopwords.words('portuguese')
stop_words_manual = ['to','pt','sobre','ca','si','al','ra','ainda','ai','so','ta','ae','fl','in','oa','fi','at','co','li','ti','qualquer','ha','dr','sa','ter','be','tal','ar','sob','onde','es','ma','of','ri','re','ir','assim','la','sempre','desde','cl','jl','is','mo','am','ac','it','oe','an','st','ro','or','en','vez','on','pa','io','dc','lo','deste','alguns','fa','pe','mi','lu','mm','va','pis','ua','af','agora','ce','le','il','he','vossa','via','pi','ne','des','ali','pr','om','quais','nesse','alguma','el','cr','az','aj','aue','ei','au','demais','nessa','av','qua','ul','ho','et','po','vo','perante','ve','pu','di','ap','tr','ie','su','ui','un','ad','per','nt','logo','go','ji','du','bo','fo','er','jo','mu',
                      'juarez', 'dia', 'gabinete', 'rua', 'data', 'telegrama', 'assunto', 'remetente', 'prezado', 'carimbo',
                     'a','ante','após','até','com','contra','de','desde','em','entre','para','por','perante','sem','sob','sobre','trás e','ou','mas','nem','logo','como','que','se','pois','porque','portanto',
                    'assaz','bastante','bem','demais','mais','menos','muito','pouco','quanto','quão','quase','tanto','tão','abaixo','acima','adiante','aí','além','ali','aquém','aqui','atrás','através','cá','defronte','dentro','detrás','fora','junto','lá','longe','onde','perto',
                    'ele','ela','nós','nos']

stop_words = set(stop_words + stop_words_manual)

def retirar_stop_words(x):
    words = [i for i in x if i not in stop_words]
    return words

df_words['Palavras'] = df_words['Palavras'].apply(retirar_stop_words) 

df_words['Palavras'] = df_words['Palavras'].apply(lambda x: ' '.join(word for word in x))

df_words = df_words[df_words['Palavras'] != '']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [ ]:
data = []
for row in df_words['Palavras'].values:
    data.append(row)

data = [i for i in data if i != '']    
#data = ' '.join(array)

In [ ]:
data_final= data

In [ ]:
data_final = []
for sentence in data[:]:
  sentence_spacy = doc(sentence)
  sentence_spacy = ' '.join([word.lemma_ for word in sentence_spacy])
  data_final.append(sentence_spacy)

A saída de streaming foi truncada nas últimas 5000 linhas.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarni

KeyboardInterrupt: ignored

In [ ]:
def bigramas(words):
 
    bigrams = []
 
    for i in range(0, len(words)):
        if (i == len(words)-1):
            break
        else:
            bigrama_obs = words[i] + '_' + words[i+1]
            bigrams.append(bigrama_obs)
 
    return bigrams

def trigramas(words):
 
    trigrams = []
 
    for i in range(0, len(words)):
        if (i == len(words)-2):
            break
        else:
            try:
                trigrama_obs = words[i] + '_' + words[i+1] + '_' + words[i+2]
                trigrams.append(trigrama_obs)
            except:
                pass
    return trigrams

In [ ]:
tokenized_data = []
texto2 = []
total_unigramas = []
total_bigramas = []
total_trigramas = []

for texto in tqdm(data_final):
    
    xtexto = word_tokenize(texto)
#CRIAR O UNIGRAMA, BIGRAMAS, TRIGRAMAS
    unigramas = xtexto
    xbigramas = bigramas(xtexto)
    xtrigramas = trigramas(xtexto)
    xunigramas = unigramas + xbigramas + xtrigramas
    
    tokenized_data.append(xunigramas)
    texto2 = texto2 + xunigramas

#CRIAR OPÇÃO PARA MOSTRAR A FREQUENCIA SOMENTE DE UNIGRAMA , BIGRAMAS E TRIGRAMAS
    total_unigramas = total_unigramas + unigramas
    total_bigramas = total_bigramas + xbigramas
    total_trigramas = total_trigramas + xtrigramas
    
xfrequencia = texto2

In [ ]:
#CRIAR DICIONÁRIO 
dictionary = corpora.Dictionary(tokenized_data)
 
#TRANSFORMAR A COLEÇÃO TEXTO COM ID
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

In [ ]:
for qtd_topicos in [5, 10, 15]:
    for qtd_palavras in tqdm([5, 10, 12, 15]):
      


        num_topics = qtd_topicos

        chunksize = 1000
        iterations = 600
        passes = 15
        eval_every = None  # Don't evaluate model perplexity, takes too much time.
        %time model = models.LdaModel(corpus=corpus, id2word=dictionary, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

        f = open(f"/content/drive/MyDrive/Projeto Juliana/Entrega 5/LDA_{qtd_topicos}_topics_{qtd_palavras}_words.txt", "w")
        f.write(f"Topics found via LDA: {qtd_topicos}_topics_{qtd_palavras}_words\n\n")

        for idx in range(num_topics):
            f.write(f"Tópico {idx}: {model.print_topic(idx, qtd_palavras)}\n\n")
            
        f.close()


        %time vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
        pyLDAvis.save_html(vis,  f'/content/drive/MyDrive/Projeto Juliana/Entrega 5/ldavis_topics_{qtd_topicos}_words_{qtd_palavras}'+'.html')

CPU times: user 3min 27s, sys: 1.16 s, total: 3min 28s
Wall time: 3min 27s
CPU times: user 7.71 s, sys: 429 ms, total: 8.14 s
Wall time: 21.7 s
CPU times: user 3min 28s, sys: 1.19 s, total: 3min 30s
Wall time: 3min 29s
CPU times: user 7.71 s, sys: 210 ms, total: 7.92 s
Wall time: 21.1 s
CPU times: user 3min 28s, sys: 1.19 s, total: 3min 29s
Wall time: 3min 28s
CPU times: user 7.7 s, sys: 176 ms, total: 7.87 s
Wall time: 20.9 s
CPU times: user 3min 28s, sys: 1.01 s, total: 3min 29s
Wall time: 3min 29s
CPU times: user 7.73 s, sys: 135 ms, total: 7.87 s
Wall time: 19.7 s



CPU times: user 4min 43s, sys: 1.33 s, total: 4min 44s
Wall time: 4min 43s
CPU times: user 9.41 s, sys: 268 ms, total: 9.68 s
Wall time: 28.5 s
CPU times: user 4min 45s, sys: 1.51 s, total: 4min 47s
Wall time: 4min 46s
CPU times: user 9.45 s, sys: 522 ms, total: 9.98 s
Wall time: 29.1 s
CPU times: user 4min 45s, sys: 1.65 s, total: 4min 47s
Wall time: 4min 46s
CPU times: user 9.41 s, sys: 246 ms, total: 9.66 s
Wall time: 29.2 s
CPU times: user 4min 44s, sys: 1.57 s, total: 4min 45s
Wall time: 4min 44s
CPU times: user 9.5 s, sys: 244 ms, total: 9.74 s
Wall time: 28.7 s



CPU times: user 6min 3s, sys: 3.28 s, total: 6min 6s
Wall time: 6min 3s
CPU times: user 12.6 s, sys: 933 ms, total: 13.5 s
Wall time: 52.6 s
CPU times: user 6min 8s, sys: 4.27 s, total: 6min 12s
Wall time: 6min 9s
CPU times: user 13 s, sys: 1.24 s, total: 14.2 s
Wall time: 51 s
CPU times: user 6min 7s, sys: 5.11 s, total: 6min 12s
Wall time: 6min 9s
CPU times: user 13.3 s, sys: 1.31 s, total: 14.6 s
Wall time: 46 s
CPU times: user 6min 7s, sys: 6.02 s, total: 6min 13s
Wall time: 6min 10s
CPU times: user 12.9 s, sys: 732 ms, total: 13.7 s
Wall time: 48.7 s



In [ ]:
 f = open(f"/content/drive/MyDrive/Projeto Juliana/Entrega 5/LDA_{3}_topics_{2}_words.txt", "w")